In [37]:
import pandas as pd
import sqlite3
import os
import json
import duckdb

In [38]:
data_path = "../data/processed/amazon_hardware_refined.jsonl"
df = pd.read_json(data_path, lines=True)

Audit whether the 1,603 rows of amazon_hardware_refined.jsonl are high-fidelity

In [39]:
# Load the refined data
data = []
with open(data_path, "r") as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

# PRINT THE AUDIT
print(f"Total Rows: {len(df)}")
print(f"Unique Brands (Stores): {df['store'].nunique()}")
print(f"Average Price: ${df['price'].mean():.2f}")
print("\n--- SAMPLE TITLES (Manual Check) ---")
print(df['title'].sample(10).values)

Total Rows: 1603
Unique Brands (Stores): 922
Average Price: $92.26

--- SAMPLE TITLES (Manual Check) ---
<ArrowStringArray>
[                                                                                                              'Great Value Cotton Fresh Automatic Spray Air Freshener Refill, 6.17 oz, (Pack of 2)',
                                                                   '16.8V-0.8A Massage Gun Charger with Adapter Charger for Muscle Massage Gun Deep Tissue 16.8V AC/DC Power Cord Fascia Gun Match…',
 'Ksquares Massage Gun Massager, Personal Handheld Percussive Deep Tissue Massage Device, Quiet Powerful Electric Massager Help Full Body Relief for Muscle Ache Pain Tension with 8 Heads and Case',
   'Massage Gun, 6 Speed High-Intensity Vibration for Athletes Deep Tissue Massager Impact Device Handheld Muscle Electric Gun, Muscle Massager Quiet Brushless Motor with 4 Heads (Anti-Skid Blue)',
                                                                   'Percussion Muscl

Determine the 50 most common words across the massage gun products:

In [40]:
from collections import Counter
import re

# 1. Tokenize the titles
# We'll grab all words, lowercase them, and ignore common "stop words"
stop_words = {'and', 'the', 'for', 'with', 'your', 'with', 'from', 'this', 'that'}
all_titles = " ".join(df['title'].tolist()).lower()
words = re.findall(r'\w+', all_titles)

# 2. Filter out short words and stop words
filtered_words = [w for w in words if len(w) > 2 and w not in stop_words]

# 3. Get the most common 50 words
word_counts = Counter(filtered_words).most_common(50)

# 4. Display as a clean DataFrame
word_freq_df = pd.DataFrame(word_counts, columns=['Word', 'Frequency'])
print(word_freq_df.head(50))

            Word  Frequency
0        massage       2271
1            gun       2226
2       massager       1930
3         muscle       1608
4           deep       1340
5         tissue       1227
6     percussion       1119
7       handheld        926
8           pain        610
9         relief        605
10          body        573
11          back        551
12         quiet        541
13      electric        512
14      athletes        505
15      portable        471
16         heads        391
17          neck        370
18      cordless        360
19         speed        290
20        speeds        261
21         black        249
22          mini        239
23     vibration        234
24        device        234
25      recovery        210
26         super        199
27  professional        185
28      shoulder        171
29      soreness        155
30         motor        154
31       therapy        152
32  rechargeable        149
33     stiffness        144
34     brushless    

# --- QUERY 1: Price and Rating Integrity Audit ---

In [41]:
con = duckdb.connect()
con.execute(f"CREATE VIEW hardware AS SELECT * FROM read_json_auto('{data_path}')")
print("--- PRICE & RATING AUDIT ---")
audit_query = """
SELECT
    count(*) AS total_items,
    count(price) AS items_with_price,
    round((count(*) - count(price)) * 100.0 / count(*), 2) AS percent_missing_price,
    round(avg(price), 2) AS avg_price,
    round(avg(rating_number), 1) AS avg_num_ratings
FROM hardware
"""
display(con.execute(audit_query).df())

--- PRICE & RATING AUDIT ---


,total_items,items_with_price,percent_missing_price,avg_price,avg_num_ratings
0,1603,252,84.28,92.26,198.0
